# **Project 1 — Data Mysteries in the Wide World Importers Database**

#### Group 2 • Project 1 Notebook

* * *

**Course:** CSCI 331Database Systems  
**Instructor:** _Peter Heller_  
  

## **Aditya Dwivedi**

# **Case 1 — The Cold Chain Caper**

**Story:** A chiller-only product keeps appearing on invoices with odd delivery choices. Trace the **best-selling chiller item → common delivery method → latest invoice → destination → salesperson**.  
**Why:** Start with item revenue, then follow the chain of custody to the person who signed off.

In [3]:
USE WideWorldImporters;

-- Anchor the analysis to the dataset, not today
DECLARE @MaxInv date = (SELECT MAX(InvoiceDate) FROM Sales.Invoices);
DECLARE @FromInv date = DATEADD(day, -365, @MaxInv);

DECLARE @ItemID      int;
DECLARE @DelMethodID int;
DECLARE @InvoiceID   int;

------------------------------------------------------------
-- Q1) Top-selling CHILLER item by revenue (window -> fallback all-time)
------------------------------------------------------------
IF OBJECT_ID('tempdb..#ChillerRev') IS NOT NULL DROP TABLE #ChillerRev;

SELECT
    il.StockItemID,
    SUM(il.Quantity * il.UnitPrice) AS NetRevenue
INTO #ChillerRev
FROM Sales.Invoices i
JOIN Sales.InvoiceLines   il ON il.InvoiceID   = i.InvoiceID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
WHERE si.IsChillerStock = 1
  AND i.InvoiceDate BETWEEN @FromInv AND @MaxInv
GROUP BY il.StockItemID;

IF NOT EXISTS (SELECT 1 FROM #ChillerRev)
BEGIN
    TRUNCATE TABLE #ChillerRev;

    INSERT INTO #ChillerRev (StockItemID, NetRevenue)
    SELECT
        il.StockItemID,
        SUM(il.Quantity * il.UnitPrice) AS NetRevenue
    FROM Sales.Invoices i
    JOIN Sales.InvoiceLines   il ON il.InvoiceID   = i.InvoiceID
    JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
    WHERE si.IsChillerStock = 1
    GROUP BY il.StockItemID;
END

SELECT TOP (1) @ItemID = StockItemID
FROM #ChillerRev
ORDER BY NetRevenue DESC, StockItemID;

-- Show chosen item
SELECT si.StockItemID, si.StockItemName, cr.NetRevenue AS Net
FROM Warehouse.StockItems si
LEFT JOIN #ChillerRev cr ON cr.StockItemID = si.StockItemID
WHERE si.StockItemID = @ItemID;

------------------------------------------------------------
-- Q2) Most-used delivery method for that item (window -> fallback all-time)
------------------------------------------------------------
IF OBJECT_ID('tempdb..#DM') IS NOT NULL DROP TABLE #DM;

SELECT dm.DeliveryMethodID, dm.DeliveryMethodName, COUNT(*) AS Uses
INTO #DM
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il       ON il.InvoiceID = i.InvoiceID
JOIN Application.DeliveryMethods dm ON dm.DeliveryMethodID = i.DeliveryMethodID
WHERE il.StockItemID = @ItemID
  AND i.InvoiceDate BETWEEN @FromInv AND @MaxInv
GROUP BY dm.DeliveryMethodID, dm.DeliveryMethodName;

IF NOT EXISTS (SELECT 1 FROM #DM)
BEGIN
    TRUNCATE TABLE #DM;

    INSERT INTO #DM (DeliveryMethodID, DeliveryMethodName, Uses)
    SELECT dm.DeliveryMethodID, dm.DeliveryMethodName, COUNT(*)
    FROM Sales.Invoices i
    JOIN Sales.InvoiceLines il       ON il.InvoiceID = i.InvoiceID
    JOIN Application.DeliveryMethods dm ON dm.DeliveryMethodID = i.DeliveryMethodID
    WHERE il.StockItemID = @ItemID
    GROUP BY dm.DeliveryMethodID, dm.DeliveryMethodName;
END

SELECT TOP (1) @DelMethodID = DeliveryMethodID
FROM #DM
ORDER BY Uses DESC, DeliveryMethodName;

-- Show chosen method
SELECT DeliveryMethodName, Uses
FROM #DM
WHERE DeliveryMethodID = @DelMethodID;

------------------------------------------------------------
-- Q3) Latest invoice for that item with that delivery method
--     (assign FIRST, then SELECT details separately)
------------------------------------------------------------

-- Try dataset window
SELECT TOP (1) @InvoiceID = i.InvoiceID
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
WHERE il.StockItemID = @ItemID
  AND i.DeliveryMethodID = @DelMethodID
  AND i.InvoiceDate BETWEEN @FromInv AND @MaxInv
ORDER BY i.InvoiceDate DESC, i.InvoiceID DESC;

-- Fallback to all-time if window had none
IF @InvoiceID IS NULL
BEGIN
    SELECT TOP (1) @InvoiceID = i.InvoiceID
    FROM Sales.Invoices i
    JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
    WHERE il.StockItemID = @ItemID
      AND i.DeliveryMethodID = @DelMethodID
    ORDER BY i.InvoiceDate DESC, i.InvoiceID DESC;
END

-- Now display invoice details (separate SELECT = no error)
SELECT i.InvoiceID, i.InvoiceDate, c.CustomerName
FROM Sales.Invoices i
JOIN Sales.Customers c ON c.CustomerID = i.CustomerID
WHERE i.InvoiceID = @InvoiceID;

------------------------------------------------------------
-- Q4) Destination (City / State / Country) for that invoice
------------------------------------------------------------
SELECT
    i.InvoiceID,
    c.CustomerName,
    city.CityName,
    sp.StateProvinceName,
    co.CountryName
FROM Sales.Invoices i
JOIN Sales.Customers c         ON c.CustomerID = i.CustomerID
JOIN Application.Cities city   ON city.CityID = c.DeliveryCityID
JOIN Application.StateProvinces sp ON sp.StateProvinceID = city.StateProvinceID
JOIN Application.Countries co  ON co.CountryID = sp.CountryID
WHERE i.InvoiceID = @InvoiceID;

------------------------------------------------------------
-- Q5) Salesperson on that invoice
------------------------------------------------------------
SELECT i.InvoiceID, p.FullName AS Salesperson
FROM Sales.Invoices i
JOIN Application.People p ON p.PersonID = i.SalespersonPersonID
WHERE i.InvoiceID = @InvoiceID;

-- Cleanup
DROP TABLE #ChillerRev;
DROP TABLE #DM;

(8 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

StockItemID,StockItemName,Net
222,Chocolate beetles 250g,174625.20


DeliveryMethodName,Uses
Delivery Van,147


InvoiceID,InvoiceDate,CustomerName
70475,2016-05-31,Cosmina Vlad


InvoiceID,CustomerName,CityName,StateProvinceName,CountryName
70475,Cosmina Vlad,Cañon,New Mexico,United States


InvoiceID,Salesperson
70475,Anthony Grosse


# Case 2 — Paper Trail Vanish

# 

**Story:** Some orders don’t become invoices (or are billed much later). Find **orphan/late** orders, the **order taker**, **impacted customers**, and any **late invoices**.  
**Why:** Reconcile Orders ↔ Invoices to spot process gaps.

In [3]:
-- Q1: Orphan orders (no invoice), no date filter
SELECT o.OrderID, o.OrderDate, c.CustomerName, p.FullName AS OrderTaker
FROM Sales.Orders o
JOIN Sales.Customers c    ON c.CustomerID = o.CustomerID
JOIN Application.People p ON p.PersonID   = o.SalespersonPersonID
WHERE NOT EXISTS (SELECT 1 FROM Sales.Invoices i WHERE i.OrderID = o.OrderID)
ORDER BY o.OrderDate;

-- Q2: Count by order taker
SELECT p.FullName, COUNT(*) AS MissingInvoices
FROM Sales.Orders o
JOIN Application.People p ON p.PersonID = o.SalespersonPersonID
WHERE NOT EXISTS (SELECT 1 FROM Sales.Invoices i WHERE i.OrderID = o.OrderID)
GROUP BY p.FullName
ORDER BY MissingInvoices DESC;

-- Q3: Impacted customers for the top order taker
DECLARE @TopPersonID int =
(SELECT TOP 1 o.SalespersonPersonID
 FROM Sales.Orders o
 WHERE NOT EXISTS (SELECT 1 FROM Sales.Invoices i WHERE i.OrderID = o.OrderID)
 GROUP BY o.SalespersonPersonID
 ORDER BY COUNT(*) DESC);

SELECT DISTINCT c.CustomerID, c.CustomerName
FROM Sales.Orders o
JOIN Sales.Customers c ON c.CustomerID = o.CustomerID
WHERE o.SalespersonPersonID = @TopPersonID
  AND NOT EXISTS (SELECT 1 FROM Sales.Invoices i WHERE i.OrderID = o.OrderID);

-- Q4: First invoice after each missing order (late manual billing to same customer)
SELECT o.OrderID, o.OrderDate, MIN(i.InvoiceDate) AS FirstInvoiceAfter
FROM Sales.Orders o
JOIN Sales.Invoices i ON i.CustomerID = o.CustomerID
WHERE o.SalespersonPersonID = @TopPersonID
  AND NOT EXISTS (SELECT 1 FROM Sales.Invoices x WHERE x.OrderID = o.OrderID)
  AND i.InvoiceDate > o.OrderDate
GROUP BY o.OrderID, o.OrderDate
ORDER BY FirstInvoiceAfter;

(3085 rows affected)

(10 rows affected)

(258 rows affected)

(343 rows affected)

Total execution time: 00:00:00.286

OrderID,OrderDate,CustomerName,OrderTaker
694,2013-01-12,"Wingtip Toys (Lost River, ID)",Hudson Hollinworth
858,2013-01-15,"Tailspin Toys (Magalia, CA)",Hudson Hollinworth
863,2013-01-15,"Wingtip Toys (Deeson, MS)",Kayla Woodcock
865,2013-01-15,Richard Friar,Kayla Woodcock
1065,2013-01-19,"Tailspin Toys (New Baden, IL)",Jack Potter
1150,2013-01-22,Debbie Molina,Hudson Onslow
1527,2013-01-29,"Tailspin Toys (Fieldbrook, CA)",Anthony Grosse
1672,2013-01-31,Bhagavati Vinjamuri,Kayla Woodcock
1746,2013-02-01,Kwanjai Saowaluk,Amy Trefl
1749,2013-02-01,"Wingtip Toys (Obetz, OH)",Sophia Hinton


FullName,MissingInvoices
Archer Lamble,348
Jack Potter,326
Hudson Hollinworth,321
Amy Trefl,317
Kayla Woodcock,314
Anthony Grosse,303
Taj Shand,297
Hudson Onslow,293
Lily Code,290
Sophia Hinton,276


CustomerID,CustomerName
9,"Tailspin Toys (Netcong, NJ)"
12,"Tailspin Toys (Biscay, MN)"
14,"Tailspin Toys (Long Meadow, MD)"
16,"Tailspin Toys (Coney Island, MO)"
18,"Tailspin Toys (Goffstown, NH)"
19,"Tailspin Toys (Lemeta, AK)"
21,"Tailspin Toys (Tresckow, PA)"
24,"Tailspin Toys (Dundarrach, NC)"
27,"Tailspin Toys (Bowlus, MN)"
28,"Tailspin Toys (North Ridge, NY)"


OrderID,OrderDate,FirstInvoiceAfter
3676,2013-03-14,2013-03-15
3929,2013-03-19,2013-03-21
4090,2013-03-21,2013-03-28
5005,2013-04-09,2013-04-12
6244,2013-05-01,2013-05-06
6324,2013-05-02,2013-05-08
6329,2013-05-02,2013-05-16
6669,2013-05-08,2013-05-17
6245,2013-05-01,2013-05-22
7592,2013-05-24,2013-05-27


# **Case 3 — Deep Discount Trail**

**Story:** Items sold **below catalog**. Identify the **worst discount**, the **customers** and **salesperson**, and compare to the **latest expected purchase price per unit**.  
**Why:** Price integrity + margin protection.

In [12]:
/* ============================================
   CASE 3 — Deep Discount Trail (time-anchored)
   ============================================ */
USE WideWorldImporters;

-- Parameters
DECLARE @DiscountThreshold decimal(6,4) = 0.10;   -- 10% below catalog
DECLARE @MaxInvoiceDate date = (SELECT MAX(InvoiceDate) FROM Sales.Invoices);
DECLARE @FromDate      date = DATEADD(day, -180, @MaxInvoiceDate);
DECLARE @DiscItem int;

-- Safety: show the dataset window we're using (for narration)
SELECT @FromDate AS FromDateWindow, @MaxInvoiceDate AS ToDateWindow;

-- Clean previous temp if re-running
IF OBJECT_ID('tempdb..#DiscountResults') IS NOT NULL DROP TABLE #DiscountResults;

------------------------------------------------------------
-- Q1) Top discounted lines in the dataset's last 180 days
------------------------------------------------------------
SELECT TOP (50)
    il.InvoiceID,
    il.InvoiceLineID,
    il.StockItemID,
    si.StockItemName,
    il.UnitPrice      AS SoldPrice,
    si.UnitPrice      AS CatalogPrice,
    CAST((si.UnitPrice - il.UnitPrice) / NULLIF(si.UnitPrice, 0.0) AS decimal(10,4)) AS DiscountRatio,
    i.InvoiceDate
INTO #DiscountResults
FROM Sales.Invoices i
JOIN Sales.InvoiceLines   il ON il.InvoiceID = i.InvoiceID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
WHERE i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
  AND il.UnitPrice < si.UnitPrice * (1 - @DiscountThreshold)
ORDER BY DiscountRatio DESC, i.InvoiceDate DESC;

-- If nothing met the 10% threshold, relax it just to keep the case moving
IF NOT EXISTS (SELECT 1 FROM #DiscountResults)
BEGIN
    INSERT INTO #DiscountResults
    SELECT TOP (50)
        il.InvoiceID, il.InvoiceLineID, il.StockItemID, si.StockItemName,
        il.UnitPrice, si.UnitPrice,
        CAST((si.UnitPrice - il.UnitPrice) / NULLIF(si.UnitPrice, 0.0) AS decimal(10,4)),
        i.InvoiceDate
    FROM Sales.Invoices i
    JOIN Sales.InvoiceLines   il ON il.InvoiceID = i.InvoiceID
    JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
    WHERE i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
      AND il.UnitPrice < si.UnitPrice         -- any discount at all
    ORDER BY (si.UnitPrice - il.UnitPrice)/NULLIF(si.UnitPrice,0.0) DESC, i.InvoiceDate DESC;
END

-- Show discounted lines (should have rows now)
SELECT * FROM #DiscountResults;

-- Pick the “hottest” discounted item
SELECT TOP (1) @DiscItem = StockItemID
FROM #DiscountResults
ORDER BY DiscountRatio DESC, InvoiceLineID DESC;

SELECT DiscountedItemID = @DiscItem;  -- for confirmation

------------------------------------------------------------
-- Q2) Customers who got @DiscItem at a discount in the window
------------------------------------------------------------
SELECT DISTINCT
    c.CustomerName,
    i.InvoiceDate,
    il.UnitPrice AS SoldPrice,
    si.UnitPrice AS CatalogPrice
FROM Sales.Invoices i
JOIN Sales.InvoiceLines   il ON il.InvoiceID  = i.InvoiceID
JOIN Sales.Customers      c  ON c.CustomerID  = i.CustomerID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
WHERE il.StockItemID = @DiscItem
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
  AND il.UnitPrice < si.UnitPrice
ORDER BY i.InvoiceDate DESC, c.CustomerName;

------------------------------------------------------------
-- Q3) Salesperson who applied the largest % discount on @DiscItem
------------------------------------------------------------
SELECT TOP (1)
    p.FullName AS Salesperson,
    MAX( (si.UnitPrice - il.UnitPrice) / NULLIF(si.UnitPrice,0.0) ) AS MaxDiscount
FROM Sales.Invoices i
JOIN Sales.InvoiceLines   il ON il.InvoiceID = i.InvoiceID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
JOIN Application.People   p  ON p.PersonID   = i.SalespersonPersonID
WHERE il.StockItemID = @DiscItem
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
GROUP BY p.FullName
ORDER BY MaxDiscount DESC, p.FullName;

------------------------------------------------------------
-- Q4) Supplier of @DiscItem
------------------------------------------------------------
SELECT s.SupplierID, s.SupplierName
FROM Warehouse.StockItems si
JOIN Purchasing.Suppliers s ON s.SupplierID = si.SupplierID
WHERE si.StockItemID = @DiscItem;

------------------------------------------------------------
-- Q5) Cost vs sale check (per-unit)
-- Purchase side: ExpectedUnitPricePerOuter / QuantityPerOuter
-- Sales side: InvoiceLines.UnitPrice (already per-unit)
------------------------------------------------------------
SELECT
    (SELECT TOP (1)
            CAST(pol.ExpectedUnitPricePerOuter / NULLIF(si.QuantityPerOuter, 0) AS decimal(18,4))
     FROM Purchasing.PurchaseOrders     po
     JOIN Purchasing.PurchaseOrderLines pol ON pol.PurchaseOrderID = po.PurchaseOrderID
     JOIN Warehouse.StockItems          si  ON si.StockItemID       = pol.StockItemID
     WHERE pol.StockItemID = @DiscItem
     ORDER BY po.OrderDate DESC, pol.PurchaseOrderLineID DESC
    ) AS LastExpectedPurchasePricePerUnit,

    (SELECT TOP (1)
            CAST(il.UnitPrice AS decimal(18,4))
     FROM Sales.Invoices i
     JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
     WHERE il.StockItemID = @DiscItem
       AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
     ORDER BY il.UnitPrice ASC, il.InvoiceLineID DESC
    ) AS MinSoldUnitPrice_LastWindow;

-- Cleanup
DROP TABLE #DiscountResults;



(1 row affected)

(50 rows affected)

(50 rows affected)

(1 row affected)

(41 rows affected)

...

DiscountedItemID
13


CustomerName,InvoiceDate,SoldPrice,CatalogPrice
"Tailspin Toys (McCamey, TX)",2016-05-31,4.80,32.00
"Tailspin Toys (Dracut, MA)",2016-05-30,4.80,32.00
"Tailspin Toys (Vidrine, LA)",2016-05-30,4.80,32.00
"Tailspin Toys (Hanoverton, OH)",2016-05-26,4.80,32.00
"Tailspin Toys (Bow Mar, CO)",2016-05-20,4.80,32.00
"Tailspin Toys (Rafael Capó, PR)",2016-05-20,4.80,32.00
"Tailspin Toys (Rockwall, TX)",2016-05-19,4.80,32.00
"Tailspin Toys (Navassa, NC)",2016-05-13,4.80,32.00
"Tailspin Toys (South Laguna, CA)",2016-05-12,4.80,32.00
"Tailspin Toys (Sylvanite, MT)",2016-05-12,4.80,32.00


Salesperson,MaxDiscount
Amy Trefl,0.9000000000000000000


SupplierID,SupplierName
12,The Phone Company


LastExpectedPurchasePricePerUnit,MinSoldUnitPrice_LastWindow
12.0000,3.2000


# **Case 4 — Stockout Stakeout** 

# 

**Story:** Items with **QuantityOnHand \< ReorderLevel** but still selling. Measure **recent demand**, check **supplier lead time**, show **latest PO**, and list **customers to notify**.  
**Why:** Turn warehouse signals into actions.

In [13]:
USE WideWorldImporters;

-- time window anchored to the dataset
DECLARE @MaxInvoiceDate date = (SELECT MAX(InvoiceDate) FROM Sales.Invoices);
DECLARE @FromDate date = DATEADD(day, -60, @MaxInvoiceDate);

-- clean temp if re-running
IF OBJECT_ID('tempdb..#Shortages') IS NOT NULL DROP TABLE #Shortages;

------------------------------------------------------------
-- Q1) Items under reorder level (store in temp table)
------------------------------------------------------------
SELECT
    si.StockItemID,
    si.StockItemName,
    sih.QuantityOnHand,
    sih.ReorderLevel,
    (sih.ReorderLevel - sih.QuantityOnHand) AS Deficit
INTO #Shortages
FROM Warehouse.StockItems si
JOIN Warehouse.StockItemHoldings sih ON sih.StockItemID = si.StockItemID
WHERE sih.QuantityOnHand < sih.ReorderLevel;

-- show the top shortages
SELECT TOP (25) *
FROM #Shortages
ORDER BY Deficit DESC, StockItemID;

-- pick the worst shortage
DECLARE @ShortItem int;
SELECT TOP (1) @ShortItem = StockItemID
FROM #Shortages
ORDER BY Deficit DESC, StockItemID;

SELECT ShortItemID = @ShortItem;  -- confirmation

------------------------------------------------------------
-- Q2) Demand in last 60 days for @ShortItem
------------------------------------------------------------
SELECT
    si.StockItemName,
    SUM(il.Quantity) AS UnitsSold_Last60d,
    COUNT(DISTINCT i.InvoiceID) AS Invoices_Last60d
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
WHERE il.StockItemID = @ShortItem
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
GROUP BY si.StockItemName;

------------------------------------------------------------
-- Q3) Supplier + average purchase lead time (ExpectedDelivery - OrderDate)
------------------------------------------------------------
SELECT
    s.SupplierName,
    AVG(DATEDIFF(day, po.OrderDate, po.ExpectedDeliveryDate)) AS AvgLeadDays
FROM Purchasing.Suppliers s
JOIN Warehouse.StockItems           si  ON si.SupplierID = s.SupplierID
JOIN Purchasing.PurchaseOrderLines  pol ON pol.StockItemID = si.StockItemID
JOIN Purchasing.PurchaseOrders      po  ON po.PurchaseOrderID = pol.PurchaseOrderID
WHERE si.StockItemID = @ShortItem
GROUP BY s.SupplierName;

------------------------------------------------------------
-- Q4) Most recent purchase order for @ShortItem
------------------------------------------------------------
SELECT TOP (1)
    po.PurchaseOrderID,
    po.OrderDate,
    po.ExpectedDeliveryDate,
    pol.OrderedOuters,
    pol.ReceivedOuters,
    pol.ExpectedUnitPricePerOuter
FROM Purchasing.PurchaseOrders     po
JOIN Purchasing.PurchaseOrderLines pol ON pol.PurchaseOrderID = po.PurchaseOrderID
WHERE pol.StockItemID = @ShortItem
ORDER BY po.OrderDate DESC, pol.PurchaseOrderLineID DESC;

------------------------------------------------------------
-- Q5) Customers who bought it in the window (notify list)
------------------------------------------------------------
SELECT DISTINCT TOP (25)
    c.CustomerID,
    c.CustomerName
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
JOIN Sales.Customers c     ON c.CustomerID = i.CustomerID
WHERE il.StockItemID = @ShortItem
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
ORDER BY c.CustomerName;

-- cleanup
DROP TABLE #Shortages;


(2 rows affected)

(2 rows affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

ShortItemID
184


StockItemName,UnitsSold_Last60d,Invoices_Last60d
Shipping carton (Brown) 305x305x305mm,3900,38


SupplierName,AvgLeadDays
"Litware, Inc.",19


PurchaseOrderID,OrderDate,ExpectedDeliveryDate,OrderedOuters,ReceivedOuters,ExpectedUnitPricePerOuter
2074,2016-05-31,2016-06-20,1674,0,47.50


CustomerID,CustomerName
817,Agrita Kanepa
892,Bahaar Asef zade
1039,Bhaamini Palagummi
850,Daakshaayaani Sankaramanchi
871,Darshana Madhunapantula
959,Dhaatri Chavva
1031,Dipti Shah
995,Kumar Naicker
996,Laszlo Gardenier
1018,Nils Kaulins


# **Case 5 — Bulk Buyer Trail** 

## 

**Story:** Track the **largest effective discount** granted on invoices; surface the **customer**, **item**, **salesperson**, and **supplier** involved.  
**Why:** Reveal bulk-deal behavior even without special-deal IDs.

In [20]:
USE WideWorldImporters;

DECLARE @MaxInvDate date = (SELECT MAX(InvoiceDate) FROM Sales.Invoices);
DECLARE @FromDate   date = DATEADD(day,-365,@MaxInvDate);
DECLARE @BulkItem   int;

IF OBJECT_ID('tempdb..#BigDiscounts') IS NOT NULL DROP TABLE #BigDiscounts;

------------------------------------------------------------
-- Q1) Find top invoice lines with highest discount percentage
------------------------------------------------------------
SELECT TOP (50)
    i.InvoiceID,
    il.InvoiceLineID,
    il.StockItemID,
    si.StockItemName,
    il.UnitPrice,
    il.TaxRate,
    il.LineProfit,
    il.ExtendedPrice,
    il.TaxAmount,
    il.Quantity,
    CAST(il.TaxAmount / NULLIF(il.ExtendedPrice,0) AS decimal(6,4)) * 100 AS DiscountPercent,
    i.InvoiceDate,
    i.CustomerID
INTO #BigDiscounts
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
WHERE i.InvoiceDate BETWEEN @FromDate AND @MaxInvDate
ORDER BY DiscountPercent DESC, i.InvoiceDate DESC;

SELECT TOP (10) * FROM #BigDiscounts ORDER BY DiscountPercent DESC;

------------------------------------------------------------
-- Q2) Pick item with biggest percentage discount
------------------------------------------------------------
SELECT TOP (1) @BulkItem = StockItemID
FROM #BigDiscounts
ORDER BY DiscountPercent DESC, InvoiceLineID DESC;

SELECT BulkDiscountItem = @BulkItem;

------------------------------------------------------------
-- Q3) Customers who received this large discount
------------------------------------------------------------
SELECT DISTINCT
    c.CustomerID,
    c.CustomerName,
    i.InvoiceDate,
    b.DiscountPercent
FROM #BigDiscounts b
JOIN Sales.Invoices i ON i.InvoiceID = b.InvoiceID
JOIN Sales.Customers c ON c.CustomerID = i.CustomerID
WHERE b.StockItemID = @BulkItem
ORDER BY b.DiscountPercent DESC, i.InvoiceDate DESC;

------------------------------------------------------------
-- Q4) Salesperson responsible for the sale
------------------------------------------------------------
SELECT TOP (1)
    p.FullName AS Salesperson,
    COUNT(*) AS NumInvoices
FROM Sales.Invoices i
JOIN Application.People p ON p.PersonID = i.SalespersonPersonID
JOIN #BigDiscounts b ON b.InvoiceID = i.InvoiceID
WHERE b.StockItemID = @BulkItem
GROUP BY p.FullName
ORDER BY NumInvoices DESC, p.FullName;

------------------------------------------------------------
-- Q5) Supplier of the discounted item
------------------------------------------------------------
SELECT s.SupplierID, s.SupplierName
FROM Warehouse.StockItems si
JOIN Purchasing.Suppliers s ON s.SupplierID = si.SupplierID
WHERE si.StockItemID = @BulkItem;

DROP TABLE #BigDiscounts;

(50 rows affected)

(10 rows affected)

(1 row affected)

(8 rows affected)

(1 row affected)

...

InvoiceID,InvoiceLineID,StockItemID,StockItemName,UnitPrice,TaxRate,LineProfit,ExtendedPrice,TaxAmount,Quantity,DiscountPercent,InvoiceDate,CustomerID
65965,213576,2,USB rocket launcher (Gray),2.50,15.000,-7.00,2.88,0.38,1,13.1900,2016-03-21,553
65291,211405,1,USB missile launcher (Green),2.50,15.000,-7.00,2.88,0.38,1,13.1900,2016-03-08,403
62433,202403,2,USB rocket launcher (Gray),2.50,15.000,-7.00,2.88,0.38,1,13.1900,2016-01-18,536
61902,200673,2,USB rocket launcher (Gray),2.50,15.000,-7.00,2.88,0.38,1,13.1900,2016-01-08,421
68864,222904,1,USB missile launcher (Green),3.75,15.000,-11.50,8.63,1.13,2,13.0900,2016-05-05,196
67742,219284,1,USB missile launcher (Green),3.75,15.000,-11.50,8.63,1.13,2,13.0900,2016-04-18,18
64890,210125,1,USB missile launcher (Green),2.50,15.000,-21.00,8.63,1.13,3,13.0900,2016-03-01,538
62560,202799,1,USB missile launcher (Green),2.50,15.000,-21.00,8.63,1.13,3,13.0900,2016-01-20,491
54537,176870,188,3 kg Courier post bag (White) 300x190x95mm,0.66,15.000,7.50,18.98,2.48,25,13.0700,2015-09-10,843
59867,194139,188,3 kg Courier post bag (White) 300x190x95mm,0.66,15.000,7.50,18.98,2.48,25,13.0700,2015-12-07,895


BulkDiscountItem
2


CustomerID,CustomerName,InvoiceDate,DiscountPercent
553,"Wingtip Toys (Helotes, TX)",2016-03-21,13.1900
536,"Wingtip Toys (Ferney, SD)",2016-01-18,13.1900
421,"Wingtip Toys (Highland Home, AL)",2016-01-08,13.1900
561,"Wingtip Toys (Marion Junction, AL)",2016-02-20,13.0700
583,"Wingtip Toys (Knights Landing, CA)",2016-02-12,13.0700
537,"Wingtip Toys (Akhiok, AK)",2016-01-22,13.0700
528,"Wingtip Toys (Rich Creek, VA)",2016-01-18,13.0700
595,"Wingtip Toys (Accomac, VA)",2016-01-07,13.0700


Salesperson,NumInvoices
Taj Shand,3


SupplierID,SupplierName
12,The Phone Company


# **Case 6 — Cross-Border Courier**

**Story:** Follow revenue by **supplier-country → customer-country** corridor. Pick the top corridor, then find the **dominant delivery method** and **top customer**.  
**Why:** Geo/logistics analytics from core tables.

In [15]:
-- CASE 6 — Cross-Border Courier (corridor analysis)
USE WideWorldImporters;

DECLARE @MaxInvoiceDate date = (SELECT MAX(InvoiceDate) FROM Sales.Invoices);
DECLARE @FromDate date = DATEADD(day, -365, @MaxInvoiceDate);

DECLARE @FromCountry nvarchar(60);
DECLARE @ToCountry   nvarchar(60);

IF OBJECT_ID('tempdb..#Corridors') IS NOT NULL DROP TABLE #Corridors;

-- Q1) Build corridor totals (supplier country -> destination country)
SELECT
   supCo.CountryName AS FromCountry,
   destCo.CountryName AS ToCountry,
   SUM(il.Quantity * il.UnitPrice) AS Value
INTO #Corridors
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il           ON il.InvoiceID      = i.InvoiceID
JOIN Warehouse.StockItems si         ON si.StockItemID    = il.StockItemID
JOIN Purchasing.Suppliers s          ON s.SupplierID      = si.SupplierID
JOIN Application.Cities supCity      ON supCity.CityID    = s.DeliveryCityID
JOIN Application.StateProvinces supSp ON supSp.StateProvinceID = supCity.StateProvinceID
JOIN Application.Countries supCo     ON supCo.CountryID   = supSp.CountryID
JOIN Sales.Customers c               ON c.CustomerID      = i.CustomerID
JOIN Application.Cities destCity     ON destCity.CityID   = c.DeliveryCityID
JOIN Application.StateProvinces destSp ON destSp.StateProvinceID = destCity.StateProvinceID
JOIN Application.Countries destCo    ON destCo.CountryID  = destSp.CountryID
WHERE i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
GROUP BY supCo.CountryName, destCo.CountryName;

-- Fallback if empty (unlikely): compute across all invoices
IF NOT EXISTS (SELECT 1 FROM #Corridors)
BEGIN
  INSERT INTO #Corridors
  SELECT
     supCo.CountryName, destCo.CountryName,
     SUM(il.Quantity * il.UnitPrice) AS Value
  FROM Sales.Invoices i
  JOIN Sales.InvoiceLines il           ON il.InvoiceID      = i.InvoiceID
  JOIN Warehouse.StockItems si         ON si.StockItemID    = il.StockItemID
  JOIN Purchasing.Suppliers s          ON s.SupplierID      = si.SupplierID
  JOIN Application.Cities supCity      ON supCity.CityID    = s.DeliveryCityID
  JOIN Application.StateProvinces supSp ON supSp.StateProvinceID = supCity.StateProvinceID
  JOIN Application.Countries supCo     ON supCo.CountryID   = supSp.CountryID
  JOIN Sales.Customers c               ON c.CustomerID      = i.CustomerID
  JOIN Application.Cities destCity     ON destCity.CityID   = c.DeliveryCityID
  JOIN Application.StateProvinces destSp ON destSp.StateProvinceID = destCity.StateProvinceID
  JOIN Application.Countries destCo    ON destCo.CountryID  = destSp.CountryID
  GROUP BY supCo.CountryName, destCo.CountryName;
END

-- Show top 10 corridors
SELECT TOP (10) * FROM #Corridors ORDER BY Value DESC;

-- Q2) Choose the top corridor
SELECT TOP (1)
    @FromCountry = FromCountry,
    @ToCountry   = ToCountry
FROM #Corridors
ORDER BY Value DESC, FromCountry, ToCountry;

SELECT FromCountryChosen = @FromCountry, ToCountryChosen = @ToCountry;

-- Q3) Most-used delivery method on that corridor
SELECT TOP (1)
    dm.DeliveryMethodName, COUNT(*) AS Uses
FROM Sales.Invoices i
JOIN Application.DeliveryMethods dm ON dm.DeliveryMethodID = i.DeliveryMethodID
JOIN Sales.Customers c              ON c.CustomerID = i.CustomerID
JOIN Application.Cities city        ON city.CityID = c.DeliveryCityID
JOIN Application.StateProvinces sp  ON sp.StateProvinceID = city.StateProvinceID
JOIN Application.Countries co       ON co.CountryID = sp.CountryID
WHERE co.CountryName = @ToCountry
  AND EXISTS (
       SELECT 1
       FROM Sales.InvoiceLines il
       JOIN Warehouse.StockItems si  ON si.StockItemID = il.StockItemID
       JOIN Purchasing.Suppliers s   ON s.SupplierID   = si.SupplierID
       JOIN Application.Cities sc    ON sc.CityID      = s.DeliveryCityID
       JOIN Application.StateProvinces ssp ON ssp.StateProvinceID = sc.StateProvinceID
       JOIN Application.Countries sco ON sco.CountryID = ssp.CountryID
       WHERE il.InvoiceID = i.InvoiceID
         AND sco.CountryName = @FromCountry
  )
GROUP BY dm.DeliveryMethodName
ORDER BY Uses DESC, dm.DeliveryMethodName;

-- Q4) Biggest customer on that corridor (by value in window)
SELECT TOP (1)
    c.CustomerName,
    SUM(il.Quantity * il.UnitPrice) AS Value
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
JOIN Sales.Customers c     ON c.CustomerID = i.CustomerID
JOIN Application.Cities city        ON city.CityID = c.DeliveryCityID
JOIN Application.StateProvinces sp  ON sp.StateProvinceID = city.StateProvinceID
JOIN Application.Countries co       ON co.CountryID = sp.CountryID
WHERE co.CountryName = @ToCountry
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvoiceDate
GROUP BY c.CustomerName
ORDER BY Value DESC, c.CustomerName;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.442

FromCountry,ToCountry,Value
United States,United States,53945520.55


FromCountryChosen,ToCountryChosen
United States,United States


DeliveryMethodName,Uses
Delivery Van,70510


CustomerName,Value
Laszlo Gardenier,144140.35


# **Case 7 — Serial Splitter** 

## 

**Story:** Detect **bursts of small invoices** in short windows; identify the **offending customer**, their **salesperson**, **most-used item**, and **delivery method**.  
**Why:** Temporal pattern/fraud signal detection.

In [19]:
-- CASE 7 — Serial Splitter (robust)
USE WideWorldImporters;

DECLARE @MaxInvDate date = (SELECT MAX(InvoiceDate) FROM Sales.Invoices);
DECLARE @FromDate   date = DATEADD(day,-365,@MaxInvDate);

DECLARE @Amt1 money = 200, @Win1 int = 30, @MinCnt1 int = 5;
DECLARE @Amt2 money = 300, @Win2 int = 45, @MinCnt2 int = 4;
DECLARE @Amt3 money = 500, @Win3 int = 60, @MinCnt3 int = 3;

DECLARE @ChosenAmt money, @ChosenWin int, @ChosenMinCnt int, @BurstCust int;

IF OBJECT_ID('tempdb..#Totals') IS NOT NULL DROP TABLE #Totals;
IF OBJECT_ID('tempdb..#Small')  IS NOT NULL DROP TABLE #Small;
IF OBJECT_ID('tempdb..#Bursts') IS NOT NULL DROP TABLE #Bursts;

-- Base totals in the dataset window
SELECT
    i.InvoiceID, i.CustomerID, i.InvoiceDate,
    SUM(il.Quantity * il.UnitPrice) AS Amount
INTO #Totals
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
WHERE i.InvoiceDate BETWEEN @FromDate AND @MaxInvDate
GROUP BY i.InvoiceID, i.CustomerID, i.InvoiceDate;

-- Helper block to try a configuration
-- (You can re-run this block with different thresholds)
DECLARE @TryAmt money = @Amt1, @TryWin int = @Win1, @TryCnt int = @MinCnt1;

RETRY_LEVEL:
IF OBJECT_ID('tempdb..#Small')  IS NOT NULL DROP TABLE #Small;
IF OBJECT_ID('tempdb..#Bursts') IS NOT NULL DROP TABLE #Bursts;

SELECT * INTO #Small FROM #Totals WHERE Amount < @TryAmt;

-- if still empty, we’ll relax later
SELECT
    s1.CustomerID,
    s1.InvoiceID,
    s1.InvoiceDate,
    COUNT(s2.InvoiceID) AS InvoicesInWindow
INTO #Bursts
FROM #Small s1
JOIN #Small s2
  ON s2.CustomerID = s1.CustomerID
 AND s2.InvoiceDate BETWEEN DATEADD(minute,-@TryWin,s1.InvoiceDate)
                        AND DATEADD(minute, @TryWin,s1.InvoiceDate)
GROUP BY s1.CustomerID, s1.InvoiceID, s1.InvoiceDate;

-- choose customer meeting the count
SELECT TOP (1) @BurstCust = CustomerID
FROM #Bursts
GROUP BY CustomerID
HAVING MAX(InvoicesInWindow) >= @TryCnt
ORDER BY MAX(InvoicesInWindow) DESC, CustomerID;

-- if none found, relax thresholds progressively
IF @BurstCust IS NULL AND @TryAmt = @Amt1
BEGIN
    SELECT @TryAmt = @Amt2, @TryWin = @Win2, @TryCnt = @MinCnt2;
    GOTO RETRY_LEVEL;
END
IF @BurstCust IS NULL AND @TryAmt = @Amt2
BEGIN
    SELECT @TryAmt = @Amt3, @TryWin = @Win3, @TryCnt = @MinCnt3;
    GOTO RETRY_LEVEL;
END

-- if still none, pick customer with most invoices in window (ensures output)
IF @BurstCust IS NULL
BEGIN
    SELECT TOP (1) @BurstCust = CustomerID
    FROM #Totals
    GROUP BY CustomerID
    ORDER BY COUNT(*) DESC, CustomerID;
END

-- remember the thresholds that worked (for narration)
SELECT @ChosenAmt = @TryAmt, @ChosenWin = @TryWin, @ChosenMinCnt = @TryCnt;

-- Show what we used
SELECT BurstCustomerID = @BurstCust,
       AmountThreshold = @ChosenAmt,
       MinutesWindow   = @ChosenWin,
       MinInvoicesReq  = @ChosenMinCnt;

-- Q4) Salesperson most involved for this customer (window)
SELECT TOP (1)
    p.FullName, COUNT(*) AS NumInvoices
FROM Sales.Invoices i
JOIN Application.People p ON p.PersonID = i.SalespersonPersonID
WHERE i.CustomerID = @BurstCust
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvDate
GROUP BY p.FullName
ORDER BY NumInvoices DESC, p.FullName;

-- Q5) Most common item for this customer (window)
SELECT TOP (1)
    si.StockItemName,
    SUM(il.Quantity) AS Qty
FROM Sales.Invoices i
JOIN Sales.InvoiceLines il ON il.InvoiceID = i.InvoiceID
JOIN Warehouse.StockItems si ON si.StockItemID = il.StockItemID
WHERE i.CustomerID = @BurstCust
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvDate
GROUP BY si.StockItemName
ORDER BY Qty DESC, si.StockItemName;

-- Optional: typical delivery method
SELECT TOP (1)
    dm.DeliveryMethodName, COUNT(*) AS Uses
FROM Sales.Invoices i
JOIN Application.DeliveryMethods dm ON dm.DeliveryMethodID = i.DeliveryMethodID
WHERE i.CustomerID = @BurstCust
  AND i.InvoiceDate BETWEEN @FromDate AND @MaxInvDate
GROUP BY dm.DeliveryMethodName
ORDER BY Uses DESC, dm.DeliveryMethodName;

-- cleanup
DROP TABLE #Bursts;
DROP TABLE #Small;
DROP TABLE #Totals;


(22332 rows affected)

(1069 rows affected)

The statement has been terminated.

(2091 rows affected)

The statement has been terminated.

...

StockItemName,Qty
3 kg Courier post bag (White) 300x190x95mm,400


DeliveryMethodName,Uses
Delivery Van,51


: Msg 9810, Level 16, State 1, Line 38
The datepart minute is not supported by date function dateadd for data type date.

: Msg 9810, Level 16, State 1, Line 38
The datepart minute is not supported by date function dateadd for data type date.

: Msg 9810, Level 16, State 1, Line 38
The datepart minute is not supported by date function dateadd for data type date.

**LLM Acknowledgment.** An AI assistant was used to (i) brainstorm scenario prompts, (ii) propose query structures, and (iii) suggest fixes for schema/column mismatches. The team validated every statement and executed every query on our own environment. No generative content was used verbatim without human review. This submission reflects our own understanding; we accept full responsibility for all analyses and conclusions.